In [16]:
---
  title: "HW 02"
  author: "INSERT YOUR NAME HERE"
  format:
    html:
      embed-resources: true
  toc: true
  jupyter: python3
  ---

SyntaxError: invalid syntax (1726070797.py, line 1)

## Step 1: Setup and Data Preprocessing

- Start by importing the necessary libraries and load the spam.csv dataset.

- Preprocess the data by encoding categorical variables, defining features and target, and splitting the data into training and testing sets. Finally, apply PCA to reduce dimensionality.

In [21]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize

In [3]:
# Load the dataset
spam = pd.read_csv("data/spam.csv")

In [6]:
# Encode categorical variables
categorical_columns = spam.select_dtypes(include = ['object', 'category']).columns.tolist()
label_encoders = {col: LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    spam[col] = label_encoders[col].fit_transform(spam[col])


In [12]:
# Define features and target
X = spam.drop('yesno', axis = 1)
y = spam['yesno']


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Reduce dimensionality
pca = PCA(n_components = 2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [55]:
# Define features and target
X = spam.drop('yesno', axis = 1)
y = spam['yesno']


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Step 2: Model Training and Decision Boundary Visualization

- Train a Decision Tree classifier on the PCA-transformed training data.

- Implement and use the `decisionplot` function (from the lecture) to visualize the decision boundary of your trained model.

In [56]:
def decisionplot(model, X, y, resolution=216):
    # Split the data into features (X) and the class variable (y)
    x_min, x_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
    y_min, y_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution),
                         np.linspace(y_min, y_max, resolution))

    # Predict outcomes for each point on the grid
    if isinstance(model, LinearDiscriminantAnalysis):
        # For LDA, we need to use the decision_function method
        Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    
    if isinstance(model, LinearDiscriminantAnalysis):
    # Reshape LDA decision function output appropriately
        Z = Z.reshape(-1, 1)
    else:
        Z = Z.reshape(xx.shape)

    # Plot the actual data points
    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, edgecolors='k', s=20)

    # Overlay the decision boundary
    plt.contourf(xx, yy, Z, alpha = 0.5)
    
    # Calculate the accuracy
    predictions = model.predict(X)
    acc = accuracy_score(y, predictions)
    
  
    # Set labels for axes
    plt.xlabel(X.columns[0])
    plt.ylabel(X.columns[1])

    plt.show()

In [57]:
#| eval: false


# Train Decision Tree
dtree = DecisionTreeClassifier()
# dtree.fit(X_train_pca, y_train)
dtree.fit(X_train, y_train)

# Implement the decisionplot function (as provided in the lecture content)
# Add the decisionplot function here

# Visualize decision boundary
# decisionplot(dtree, pd.DataFrame(X_train_pca, columns = ['PC1', 'PC2']), y_train)
# decisionplot(dtree, X_train, y_train)



DecisionTreeClassifier()

## Step 3: Model Evaluation

- Evaluate your model using accuracy, precision, recall, F1 score, and AUC-ROC metrics.

In [58]:
#| eval: false


# Predictions
# predictions = dtree.predict(X_test_pca)
predictions = dtree.predict(X_test)

# Evaluate metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average = 'weighted')
recall = recall_score(y_test, predictions, average = 'weighted')
f1 = f1_score(y_test, predictions, average = 'weighted')

# Display results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# For AUC-ROC, binarize the output and calculate AUC-ROC for each class
# Add the necessary code for AUC-ROC calculation here (refer to lecture content)

Accuracy: 0.83
Precision: 0.83
Recall: 0.83
F1 Score: 0.83


In [44]:
# Binarize the output for multiclass
y_test_binarized = label_binarize(y_test, classes = np.unique(y_train))
n_classes = y_test_binarized.shape[1]

# Get the probability predictions for each class
y_score = dtree.predict_proba(X_test_pca)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calculate macro-average ROC-AUC
roc_auc_macro = np.mean(list(roc_auc.values()))
print(f"Macro-average ROC-AUC: {roc_auc_macro:.2f}")

# Calculate micro-average ROC-AUC
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_score[:,1].ravel())
roc_auc_micro = auc(fpr["micro"], tpr["micro"])
print(f"Micro-average ROC-AUC: {roc_auc_micro:.2f}")

Macro-average ROC-AUC: 0.23
Micro-average ROC-AUC: 0.77


In [43]:
y_score

array([[0.95652174, 0.04347826],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.66666667, 0.33333333],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [28]:
X_test_pca.shape

(921, 2)

In [32]:
y_test.shape

(921,)

In [30]:
y_score.shape

(921, 2)

In [33]:
y_score

array([[0.95652174, 0.04347826],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.66666667, 0.33333333],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [25]:
y_test_binarized.shape

(921, 1)

## Assignment:
- Implement the missing parts of the code: the decisionplot function and AUC-ROC calculation.

- Discuss the results among your peers. Consider the following:

    - Which metric is most informative for this problem and why? It's surprising that the four Accuracy, Precision,Recall, and F1 Score yield the same value, so I prefer to use ROC-AUC metric.

    - How does the decision boundary visualization help in understanding the model’s performance? In fact, the decision boundary just shows a very general view of how the data can be visualized, but it is not helpful. Also, when we use all the variables and we have more than 3 dimension, the decision boundary can't show anything. 

    - Reflect on the impact of PCA on model performance and decision boundary.
    Not using PCA yield higher classification performance! All the metrics were increased!
